## This laboratory will be conducted over two sessions
## You are expected to finish the software and test the code in simulation between the sessions! ##

# Manipulating objects

For this laboratory, we will use two robots distant from each other of 25cm as depicted in the figure below

<img src="./2_robot_config.png" width="500">

There is 1cm between the end-effector "shell" and the position of the frame as depicted in the figure below

<img src="./foot_close.png" width="300">

<img src="./box.jpg" width="200">

The objects to grasp are soft cubes with a 5cm side.
Three cubes are stacked on top of each other. The goal of the laboratory is to design a controller to will move the two cubes on the top away
and pick the bottom cube and lift it up by 10cm.

In [1]:
#setup nice plotting (use widget instead of notebook in the command below if you use jupyter lab)

# we import useful libraries
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
from time import sleep

# this function returns the body Jacobian
from robot_utils import getBodyJacobianOrientedLikeSpatialFrame

# we import the robot class
use_real_robot = False

if use_real_robot:
    from NYUFinger.sim import NYUDualFingerReal as Robot   
else:
    from NYUFinger.sim import NYUDualFingerSimulator as Robot


# here we define the global variables for the robot size
l0 = 0.3
l1 = 0.16
l2 = 0.16
l3 = 0.014

## Instantiate The Robots
If you're using the real robots, put them into zero position and then run the next cell to instantiate and reset them. 

In [2]:
robot = Robot()
if use_real_robot:
    robot.reset_sensors()
else:
    robot.reset()

## Interacting with the Robot
For this lab, you'll be using two robots at the same time. Unlike the previous labs where the $\tau$, $q$, and $dq$ were 3D vectors, in this lab robot takes 6D commands and returns 6D states. The first three numbers are for finger 1 (left) and the last three are for the finger 2 (right). Let's make sure you can read the sates and command the robot:

In [ ]:
q, dq = robot.get_state()
q1 = q[:3]
q2 = q[3:]
dq1 = dq[:3]
dq2 = dq[3:]
print(f'Robot1: q={q1}, dq1={dq1}')
print(f'Robot2: q={q2}, dq1={dq2}')

If you're using the simulator, you can also access the ground truth posiion of the cubes using `robot.get_cube_positions()` and reset their location using `robot.reset()`: 

In [ ]:
if not use_real_robot:
    robot.reset()
    print(robot.get_cube_positions())

## Questions
In this laboratory you are expected to re-use code from the previous laboratories (e.g. to plot, compute forward kinematics, etc) and solve the problem in both simulation and using the real robot. For both simulation and real robot deployment, you're expected to upload a short video of the robot solving the task alongside the report containing plots showing the robot reaching the desired 10cm goal for the designated cube.

1. Using an impedance controller (Lab4 - use the ``getBodyJacobianOrientedLikeSpatialFrame`` function to get the Jacobian) and one trajectory generation method seen in Lab3 write a complete controller that solves the problem: move the top two cubes away, grasp the last cube and move it up by 10cm. In the report, describe in details the controller you designed (what it does, etc) and the results you obtained.

2. Test your controller in simulation. Save and plot the results.
   
3. Test the controller on the real robots. Save and plot the results.

4. (optional - bonus) Extend the controller to rearrange the cubes - putting the bottom one on top of the two others.

In [ ]:
# we simulate for 10 seconds
run_time = 13.
dt = 0.001
num_steps = int(run_time/dt)

# the PD gains
P = np.array([1.5, 1.5, 1.5])
D = np.array([0.05, 0.05, 0.05])


## we allocate a few vectors to collect data while we do the simulation
# we keep track of time
time = np.zeros([num_steps])
# we will store the sensor values (angular position and velocities)
measured_positions_f1 = np.zeros([num_steps,3])
measured_velocities_f1 = np.zeros_like(measured_positions_f1)
measured_positions_f2 = np.zeros_like(measured_positions_f1)
measured_velocities_f2 = np.zeros_like(measured_positions_f1)
# we will store the end effector position (desired and measured)
measured_finger_position_f1 = np.zeros([num_steps,3])
desired_finger_position_f1 = np.zeros([num_steps,3])
measured_finger_position_f2 = np.zeros([num_steps,3])
desired_finger_position_f2 = np.zeros([num_steps,3])


for i in range(num_steps):
    # get the current time and save it
    time[i] = dt * i
    
    # we get the position and velocities of the joints
    q, dq = robot.get_state()
    q1 = q[:3]
    q2 = q[3:]
    dq1 = dq[:3]
    dq2 = dq[3:]
    
    # controller: TODO HERE IMPLEMENT YOUR CONTROLLER

    joint_torques1 = np.zeros([3,1])
    joint_torques2 = np.zeros([3,1])
    joint_torques = np.vstack([joint_torques1, joint_torques2]).squeeze()
    # we send them to the robot and do one simulation step
    robot.send_joint_torque(joint_torques)
    sleep(dt)